## Vector Search with IRIS SQL
This tutorial covers how to use IRIS as a vector database. 

For this tutorial, we will use a dataset of 2.2k online reviews of scotch (
dataset from https://www.kaggle.com/datasets/koki25ando/22000-scotch-whisky-reviews) . With our latest vector database functionality, we can leverage the latest embedding models to run semantic search on the online reviews of scotch whiskeys. In addition, we'll be able to apply filters on columns with structured data. For example, we will be able to search for whiskeys that are priced under $100, and are 'earthy, smooth, and easy to drink'. Let's find our perfect whiskey!

In [1]:
import os, pandas as pd
from sentence_transformers import SentenceTransformer
import iris 

/Users/aryanput/dp/hackathon-2024/venv/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
username = 'demo'
password = 'demo'
hostname = os.getenv('IRIS_HOSTNAME', 'localhost')
port = '1972' 
namespace = 'USER'
CONNECTION_STRING = f"{hostname}:{port}/{namespace}"

In [3]:
# Note: Ideally conn and cursor should be used with context manager or with try-execpt-finally 
conn = iris.connect(CONNECTION_STRING, username, password)
cursor = conn.cursor()

In [4]:
# Load the CSV file
df = pd.read_csv('../data/scotch_review.csv')

In [5]:
# View the data
df.head()

,Unnamed: 0,name,category,review.point,price,currency,description
0,1,"Johnnie Walker Blue Label, 40%",Blended Scotch Whisky,97,225,$,"Magnificently powerful and intense. Caramels, ..."
1,2,"Black Bowmore, 1964 vintage, 42 year old, 40.5%",Single Malt Scotch,97,4500.00,$,What impresses me most is how this whisky evol...
2,3,"Bowmore 46 year old (distilled 1964), 42.9%",Single Malt Scotch,97,13500.00,$,There have been some legendary Bowmores from t...
3,4,"Compass Box The General, 53.4%",Blended Malt Scotch Whisky,96,325,$,With a name inspired by a 1926 Buster Keaton m...
4,5,"Chivas Regal Ultis, 40%",Blended Malt Scotch Whisky,96,160,$,"Captivating, enticing, and wonderfully charmin..."


In [6]:
# Clean data
# Remove the specified columns
df.drop(['currency'], axis=1, inplace=True)

# Drop the first column
df.drop(columns=df.columns[0], inplace=True)

# Remove rows without a price
df.dropna(subset=['price'], inplace=True)

# Ensure values in 'price' are numbers
df = df[pd.to_numeric(df['price'], errors='coerce').notna()]

# Replace NaN values in other columns with an empty string
df.fillna('', inplace=True)

In [7]:
# View cleaned data
df.head()

,name,category,review.point,price,description
0,"Johnnie Walker Blue Label, 40%",Blended Scotch Whisky,97,225,"Magnificently powerful and intense. Caramels, ..."
1,"Black Bowmore, 1964 vintage, 42 year old, 40.5%",Single Malt Scotch,97,4500.00,What impresses me most is how this whisky evol...
2,"Bowmore 46 year old (distilled 1964), 42.9%",Single Malt Scotch,97,13500.00,There have been some legendary Bowmores from t...
3,"Compass Box The General, 53.4%",Blended Malt Scotch Whisky,96,325,With a name inspired by a 1926 Buster Keaton m...
4,"Chivas Regal Ultis, 40%",Blended Malt Scotch Whisky,96,160,"Captivating, enticing, and wonderfully charmin..."


In [11]:
# Load a pre-trained sentence transformer model. This model's output vectors are of size 384
model = SentenceTransformer('all-MiniLM-L6-v2') 

In [12]:
# Generate embeddings for all descriptions at once. Batch processing makes it faster
embeddings = model.encode(df['description'].tolist(), normalize_embeddings=True)

# Add the embeddings to the DataFrame
df['description_vector'] = embeddings.tolist()


Now, InterSystems IRIS supports vectors as a datatype in tables! Here, we create a table with a few different columns. The last column, 'description_vector', will be used to store vectors that are generated by passing the 'description' of a review through an embedding model.

In [10]:
# Create SQL table
sql = """
        CREATE TABLE scotch_reviews (
name VARCHAR(255),
category VARCHAR(255),
review_point INT,
price DOUBLE,
description VARCHAR(2000),
description_vector VECTOR(DOUBLE, 384)
)
        """
result = cursor.execute(sql)

In [ ]:
# To delete the SQL table:
# sql = "DROP TABLE scotch_reviews"
# result = cursor.execute(sql)

To execute SQL statements, use:
```python 
cursor.execute(< sql statement >, < list of parameters in sql statement >)
cursor.executemany(< sql statement>, < list of tuples, where each tuple represents parameters to insert into each row>)
```
cursor.executemany is going to be more efficient, especially for larger datasets. But, either way works!


In [26]:
# Prepare SQL query
sql = """
    INSERT INTO scotch_reviews
    (name, category, review_point, price, description, description_vector) 
    VALUES (?, ?, ?, ?, ?, TO_VECTOR(?))
"""

# Prepare the list of tuples (parameters for each row)
data = [
    (
        row['name'], 
        row['category'], 
        row['review.point'], 
        row['price'], 
        row['description'], 
        str(row['description_vector']) 
    )
    for index, row in df.iterrows()
]
cursor.executemany(sql, data)

(1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [31]:
# Alternatively, use cursor.execute to insert each row one at a time

# for index, row in df.iterrows():
#     # Prepare the parameters for each row
#     params = [
#         row['name'], 
#         row['category'], 
#         row['review.point'], 
#         row['price'], 
#         row['description'], 
#         str(row['description_vector'])]
    
    
#     # Execute the SQL statement for each row
#     cursor.execute(sql, params)

Let's look for a scotch that costs less than $100, and has an earthy and creamy taste.

In [33]:
# This is our search phrase
description_search = "earthy and creamy taste"

# Convert search phrase into a vector
search_vector = model.encode(description_search, normalize_embeddings=True).tolist() 

In [32]:
# Define the SQL query with placeholders for the vector and limit
sql = """
    SELECT TOP ? *
    FROM scotch_reviews
    WHERE price < 100 
    ORDER BY VECTOR_DOT_PRODUCT(description_vector, TO_VECTOR(?)) DESC
"""

numberOfResults = 3

# Execute the query with the number of results and search vector as parameters
cursor.execute(sql, [numberOfResults, str(search_vector)])

# Fetch all results
results = cursor.fetchall()


In [29]:
print(results)

(('Signatory (distilled at Bowmore), 16 year old, 1988 vintage, cask #42508, 46%', 'Single Malt Scotch', 87, 60.0, 'Medium-bodied and nicely textured. Good balance of flavors -- and well-integrated, too -- with lovely sweet notes (cereal grain, cookie dough, caramel, and vanilla cream), young heathery peat, tar, fishnets, and brine that is complementary, but not aggressive, with a suggestion of lavender and tangerine. Balanced finish. (332 bottles produced.)', '-.048620302230119705201,-.082065835595130920411,.039660654962062835693,-.018970942124724388122,-.017485225573182106018,.042453143745660781861,.046325396746397018432,.0010440859477967023849,-.044980168342590332031,-.029944123700261116027,.0052331099286675453186,-.061179354786872863769,-.067733250558376312256,-.067550390958786010742,.037490777671337127686,.031035721302032470703,.14657463133335113526,-.011625745333731174468,.015587563626468181611,.0036643501371145248413,.0020548028405755758286,.051617961376905441284,.05884261056780

In [30]:
results_df = pd.DataFrame(results, columns=df.columns).iloc[:, :-1] # Remove vector
pd.set_option('display.max_colwidth', None)  # Easier to read description
results_df.head()

,name,category,review.point,price,description
0,"Signatory (distilled at Bowmore), 16 year old, 1988 vintage, cask #42508, 46%",Single Malt Scotch,87,60.0,"Medium-bodied and nicely textured. Good balance of flavors -- and well-integrated, too -- with lovely sweet notes (cereal grain, cookie dough, caramel, and vanilla cream), young heathery peat, tar, fishnets, and brine that is complementary, but not aggressive, with a suggestion of lavender and tangerine. Balanced finish. (332 bottles produced.)"
1,"Signatory (distilled at Bowmore), 16 year old, 1988 vintage, cask #42508, 46%",Single Malt Scotch,87,60.0,"Medium-bodied and nicely textured. Good balance of flavors -- and well-integrated, too -- with lovely sweet notes (cereal grain, cookie dough, caramel, and vanilla cream), young heathery peat, tar, fishnets, and brine that is complementary, but not aggressive, with a suggestion of lavender and tangerine. Balanced finish. (332 bottles produced.)"
2,"Shieldaig 12 year old, 40%",Blended Scotch Whisky,85,31.0,"This is a sharp dresser, with a firm, solid mouthfeel and an altogether finer and more focused taste than Shieldaig Classic (see\r\nbelow). It’s not coastal or earthy particularly, either. Instead the flavors are softer and built around mocha, smooth creamy toffee, and some soft fruit, including a touch of overripe banana and melon notes. The savoriness this time comes from a touch of pepper rather than salt."
